In [1]:
import sys
sys.path.append("../agrilearn/submodules/commons/")
from agrilearn.commons.s3 import s3_utils
from agrilearn.crop_classification import evaluate_utils, yaml_utils, processing
import geopandas as gpd
import os
import pandas as pd

2025-04-17 20:48:22.434326: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 20:48:22.449802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744922902.469118  325975 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744922902.474968  325975 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 20:48:22.494620: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
#config = yaml_utils.load_config("/agrilearn_app/output/config/experiment_12.yaml")

In [2]:
MAIN_PATH = "/agrilearn_app/datasets/CORN/2023_2023/"
GEOPACKAGE_FILE_PATH = os.path.join(MAIN_PATH, "geopackage/processed/CORN_33026.gpkg")
NEW_EOPATCH_PATH = os.path.join(MAIN_PATH, "eopatch/processed/")
REPORT_PATH = os.path.join(MAIN_PATH, "reports/")
OLD_EOPATCH_PATH = "/agrilearn_app/datasets/CORN/eopatch/processed/"

LABEL_EOPATCH_LOCATION = "eopath_location" 

In [3]:
df = gpd.read_file(GEOPACKAGE_FILE_PATH)

In [4]:
df.shape

(33026, 73)

In [5]:
df.head()

,monitoring_class,period,fonte,state,area,micro,eopath_location,start_season,end_season,peak_start,...,obs_emergence,obs_senescence,obs_harvest,created_by,area_id,last_date_crop_rnn,crop_distance_score_sits,crop_confidence_maha_sits,set,geometry
0,CORN,2023/2023,Agrosatélite - Grãos 2022/2023,RO,None,0.0,start_2023-01-01_end_2023-09-01_monitoring_cla...,2023-01-01,2023-09-01,2023-03-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-63.43957 -9.29016, -63.43957 ..."
1,CORN,2023/2023,Agrosatélite - Grãos 2022/2023,RO,871451.0441404884,0.0,start_2023-01-01_end_2023-09-01_monitoring_cla...,2023-01-01,2023-09-01,2023-03-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-63.42025 -9.27824, -63.42025 ..."
2,CORN,2023/2023,Agrosatélite - Grãos 2022/2023,RO,1604076.711068449,0.0,start_2023-01-01_end_2023-09-01_monitoring_cla...,2023-01-01,2023-09-01,2023-03-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-63.41611 -9.27407, -63.41611 ..."
3,CORN,2023/2023,Agrosatélite - Grãos 2022/2023,RO,1496776.078012722,0.0,start_2023-01-01_end_2023-09-01_monitoring_cla...,2023-01-01,2023-09-01,2023-03-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-63.4313 -9.29645, -63.4317 -9..."
4,CORN,2023/2023,Agrosatélite - Grãos 2022/2023,RO,1486526.409996468,0.0,start_2023-01-01_end_2023-09-01_monitoring_cla...,2023-01-01,2023-09-01,2023-03-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-63.40514 -9.29191, -63.40515 ..."


In [6]:
df_results = processing.check_load_eopatch_and_change_eopatch_dir(df,
                                        eopatch_path=OLD_EOPATCH_PATH,
                                        new_eopatch_path=NEW_EOPATCH_PATH,
                                        label_monitoring_class='monitoring_class',
                                        label_eopatch_path='eopath_location')

  0%|          | 0/33026 [00:00<?, ?it/s]

## Check eopatch

In [7]:
df_processed, df_overview_processed = processing.check_processed_eopatches_features_based_geopackage(df, 
                                                                                         eopatch_path=NEW_EOPATCH_PATH,
                                                                                         label_eopatch_path=LABEL_EOPATCH_LOCATION)

  0%|          | 0/33026 [00:00<?, ?it/s]

In [8]:
df_processed[df_processed['eopatch_processed_exists']==False]

,eopatch_location_id,eopatch_processed_exists,len_data,len_bands,len_time_interval,len_timestamp,error_processed


## Merge Data

In [ ]:
df['eopath_location'].loc[0]

In [ ]:
df_processed['eopatch_location_id'].loc[0]

In [ ]:
# Supondo que df e df_processed já estejam definidos
df_full = pd.merge(df, 
                   df_processed,
                   left_on='eopath_location',
                   right_on='eopatch_location_id',
                   how='left')

In [ ]:
duplicacoes = df_full[df_full.duplicated(subset=['eopath_location'], keep=False)]

In [ ]:
df_full['eopath_location'].nunique()

## Save data

In [ ]:
result_string= '_'.join([f"{cls}_{count}" for cls, count in df_full['monitoring_class'].value_counts().items()])
result_string

In [ ]:
os.makedirs(REPORT_PATH, exist_ok=True)
df_full.to_csv(f"{REPORT_PATH}/overview_{result_string}.csv")

In [ ]:
df_full.to_file(GEOPACKAGE_FILE_PATH, 
                driver='GPKG', 
                engine='fiona')

In [ ]:
# GEOPACKAGE_PATH = "/agrilearn_app/datasets/baselines/testes_de_concordancia/geopackage/SOYBEAN_216_CORN_127.gpkg"
# EOPATCH_PATH = "/agrilearn_app/datasets/baselines/teste_pre_safra_2024_2025/eopatch/processed/"
# NEW_EOPATCH_PATH = "/agrilearn_app/datasets/baselines/testes_de_concordancia/eopatch/processed/"

In [ ]:
# df = gpd.read_file(GEOPACKAGE_PATH)

In [ ]:
# df

In [ ]:
# df_results_v2 = processing.check_load_eopatch_and_change_eopatch_dir(df,
#                                         eopatch_path=EOPATCH_PATH,
#                                         new_eopatch_path=NEW_EOPATCH_PATH,
#                                         label_monitoring_class='monitoring_class',
#                                         label_eopatch_path='eopath_location')